# Deterministic Testing for Agents

Testing agents without calling real LLMs using ModelMock and tool_mock.
This enables fast, reliable, and deterministic tests.

In [ ]:
from pydantic_ai.messages import ToolCallPart

from agentic_patterns.core.agents import get_agent, run_agent
from agentic_patterns.testing import ModelMock, tool_mock

## Testing with ModelMock

ModelMock replaces the LLM with predefined responses.
The agent runs normally, but model responses are deterministic.

In [ ]:
model = ModelMock(responses=["The capital of France is Paris."])
agent = get_agent(model=model)

agent_run, _ = await run_agent(agent, "What is the capital of France?")
output = agent_run.result.output

print(f"Output: {output}")
assert output == "The capital of France is Paris."

## Testing Tool Calls

ModelMock can return ToolCallPart to simulate the model calling tools.
The sequence: model calls tool -> tool executes -> model responds with final answer.

In [ ]:
def get_weather(city: str) -> str:
    """Get weather for a city."""
    return f"Weather in {city}: 22C, sunny"

model = ModelMock(responses=[
    ToolCallPart(tool_name="get_weather", args={"city": "Paris"}),
    "The weather in Paris is 22C and sunny."
])
agent = get_agent(model=model, tools=[get_weather])

agent_run, nodes = await run_agent(agent, "What's the weather in Paris?", verbose=True)
output = agent_run.result.output

assert "22C" in output
assert "sunny" in output

## Mocking Tools with tool_mock

tool_mock replaces tool implementations with predefined return values.
Useful for testing agent behavior with specific tool outputs.

In [ ]:
def fetch_stock_price(symbol: str) -> float:
    """Fetch current stock price."""
    raise NotImplementedError("Real implementation would call API")

# Create mock that returns predefined prices
mocked_fetch = tool_mock(fetch_stock_price, return_values=[150.25, 2800.50])

model = ModelMock(responses=[
    ToolCallPart(tool_name="fetch_stock_price", args={"symbol": "AAPL"}),
    ToolCallPart(tool_name="fetch_stock_price", args={"symbol": "GOOG"}),
    "AAPL is at $150.25, GOOG is at $2800.50."
])
agent = get_agent(model=model, tools=[mocked_fetch])

agent_run, _ = await run_agent(agent, "Compare AAPL and GOOG prices", verbose=True)

print(f"Tool was called {mocked_fetch.call_count} times")
print(f"Call arguments: {mocked_fetch.call_args_list}")

assert mocked_fetch.call_count == 2
assert mocked_fetch.call_args_list[0] == ((), {"symbol": "AAPL"})
assert mocked_fetch.call_args_list[1] == ((), {"symbol": "GOOG"})

## Full Deterministic Test

Combining ModelMock and tool_mock for complete control over agent behavior.
This pattern enables testing complex multi-step workflows.

In [ ]:
def search_database(query: str) -> list[str]:
    """Search database for matching records."""
    raise NotImplementedError()

def send_email(to: str, subject: str, body: str) -> bool:
    """Send an email."""
    raise NotImplementedError()

# Mock both tools
mocked_search = tool_mock(search_database, return_values=[["user@example.com", "admin@example.com"]])
mocked_email = tool_mock(send_email, return_values=[True, True])

# Define the expected agent behavior
model = ModelMock(responses=[
    ToolCallPart(tool_name="search_database", args={"query": "active users"}),
    [ToolCallPart(tool_name="send_email", args={"to": "user@example.com", "subject": "Update", "body": "Hello"}),
     ToolCallPart(tool_name="send_email", args={"to": "admin@example.com", "subject": "Update", "body": "Hello"})],
    "Sent emails to 2 users."
])

agent = get_agent(model=model, tools=[mocked_search, mocked_email])
agent_run, _ = await run_agent(agent, "Email all active users with an update", verbose=True)

# Verify the workflow executed correctly
assert mocked_search.call_count == 1
assert mocked_email.call_count == 2
assert "2 users" in agent_run.result.output

print("Test passed: workflow executed as expected")